# Context

It's not that we humans only take debts to manage our necessities. A country may also take debt to manage its economy. For example, infrastructure spending is one costly ingredient required for a country's citizens to lead comfortable lives. [The World Bank](https://www.worldbank.org) is the organization that provides debt to countries.

In this notebook, we are going to analyze international debt data collected by The World Bank. The dataset contains information about the amount of debt (in USD) owed by developing countries across several categories. We are going to find the answers to questions like: 

- What is the total amount of debt that is owed by the countries listed in the dataset?
- Which country owns the maximum amount of debt and what does that amount look like?
- What is the average amount of debt owed by countries across different debt indicators?

In [22]:
# importing libraries

import pandas as pd
import sqlite3

In [23]:
# installing the ipython-sql library

!pip install ipython-sql

In [24]:
# creating dataframe to import data
international_debt = pd.read_csv("international_debt.csv")
international_debt.head()

,country_name,country_code,indicator_name,indicator_code,debt
0,Afghanistan,AFG,"Disbursements on external debt, long-term (DIS...",DT.DIS.DLXF.CD,72894453.7
1,Afghanistan,AFG,"Interest payments on external debt, long-term ...",DT.INT.DLXF.CD,53239440.1
2,Afghanistan,AFG,"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD,61739336.9
3,Afghanistan,AFG,"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD,49114729.4
4,Afghanistan,AFG,"PPG, bilateral (INT, current US$)",DT.INT.BLAT.CD,39903620.1


In [25]:
#creating a connection

cnn = sqlite3.connect('world_bank_debt.db')

In [28]:
international_debt.to_sql('world_debt',cnn)
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [29]:
%sql sqlite:///world_bank_debt.db

In [35]:
%%sql
SELECT COUNT(DISTINCT(country_name)) AS total_distinct_countries
FROM world_debt;

   sqlite:///wb_debt.db
 * sqlite:///world_bank_debt.db
Done.


total_distinct_countries
124


In [40]:
%%sql
SELECT DISTINCT(indicator_name) AS distinct_debt_indicators
FROM world_debt
ORDER BY distinct_debt_indicators;

   sqlite:///wb_debt.db
 * sqlite:///world_bank_debt.db
Done.


distinct_debt_indicators
"Disbursements on external debt, long-term (DIS, current US$)"
"Interest payments on external debt, long-term (INT, current US$)"
"Interest payments on external debt, private nonguaranteed (PNG) (INT, current US$)"
"PPG, bilateral (AMT, current US$)"
"PPG, bilateral (DIS, current US$)"
"PPG, bilateral (INT, current US$)"
"PPG, bonds (AMT, current US$)"
"PPG, bonds (INT, current US$)"
"PPG, commercial banks (AMT, current US$)"
"PPG, commercial banks (DIS, current US$)"


In [43]:
%%sql
SELECT ROUND(SUM(debt / 1000000), 2) AS total_debt
FROM world_debt;

   sqlite:///wb_debt.db
 * sqlite:///world_bank_debt.db
Done.


total_debt
3079734.49


In [44]:
%%sql

SELECT country_name, SUM(debt) AS total_debt 
FROM world_debt
GROUP BY country_name
ORDER BY total_debt DESC
LIMIT 1;

   sqlite:///wb_debt.db
 * sqlite:///world_bank_debt.db
Done.


country_name,total_debt
China,285793494734.2


In [23]:
--avg_debt_per_indicator
SELECT indicator_code AS debt_indicator, indicator_name, AVG(debt) AS average_debt 
FROM international_debt
GROUP BY debt_indicator, indicator_name
ORDER BY average_debt DESC
LIMIT 10;


,debt_indicator,indicator_name,average_debt
0,DT.AMT.DLXF.CD,"Principal repayments on external debt, long-te...",5.904868e+09
1,DT.AMT.DPNG.CD,"Principal repayments on external debt, private...",5.161194e+09
2,DT.DIS.DLXF.CD,"Disbursements on external debt, long-term (DIS...",2.152041e+09
3,DT.DIS.OFFT.CD,"PPG, official creditors (DIS, current US$)",1.958983e+09
4,DT.AMT.PRVT.CD,"PPG, private creditors (AMT, current US$)",1.803694e+09
5,DT.INT.DLXF.CD,"Interest payments on external debt, long-term ...",1.644024e+09
6,DT.DIS.BLAT.CD,"PPG, bilateral (DIS, current US$)",1.223139e+09
7,DT.INT.DPNG.CD,"Interest payments on external debt, private no...",1.220411e+09
8,DT.AMT.OFFT.CD,"PPG, official creditors (AMT, current US$)",1.191188e+09
9,DT.AMT.PBND.CD,"PPG, bonds (AMT, current US$)",1.082624e+09


In [48]:
%%sql 

SELECT world_debt.country_name, world_debt.indicator_name 
FROM world_debt
WHERE debt = (SELECT MAX(debt) 
			  FROM world_debt
			  WHERE indicator_code = 'DT.AMT.DLXF.CD');

   sqlite:///wb_debt.db
 * sqlite:///world_bank_debt.db
Done.


country_name,indicator_name
China,"Principal repayments on external debt, long-term (AMT, current US$)"
